# Hyperparameter Tuning using HyperDrive

In [1]:
import azureml
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Model, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib
import pandas as pd

In [2]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="lol_match_predictor")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FXT2UW2KU to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-136182
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-136182


In [3]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
dataset = pd.read_csv("https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20210126.csv")

# Show raw data before it is processed.
dataset.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,118.0,165.0,166.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,98.0,-399.0,150.0,-7.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,140.0,-409.0,-1837.0,-11.0,0.0,0.0,1.0,0.0,1.0,0.0
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,135.0,51.0,-401.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,28.0,-233.0,257.0,-8.0,0.0,0.0,0.0,0.0,1.0,0.0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,40,60,80,100,150,200,300,400,600,800)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(".", compute_target=compute_target, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                     estimator=estimator,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)






'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.9069412931235706', '--max_iter', '800']


In [9]:
#TODO: Save the best model

joblib.dump(value=best_run.id, filename="model.pkl")

# Register best model
model = Model.register(model_path = "model.pkl",
                       model_name = "lol-predictor",
                       description = "League of Legends match predictor trained using Azure ML.",
                       workspace = ws)

Registering model lol-predictor


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [99]:
# Get environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Create inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Set deployment configuration as an Azure Container Instance. Note logging is enabled
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

# Service deployment
service = Model.deploy(
    workspace=ws,
    name="lolpredict",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [84]:
print(service.get_logs())

None


TODO: In the cell below, send a request to the web service you deployed to test it.

In [105]:
scoring_uri = service.scoring_uri

print("Service State: {}".format(service.state))
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))


Service State: Healthy
Scoring URI: http://34804073-54a0-4518-861d-03630321db25.southcentralus.azurecontainer.io/score
Swagger URI: http://34804073-54a0-4518-861d-03630321db25.southcentralus.azurecontainer.io/swagger.json


In [109]:
import requests
import json

scoring_uri=service.scoring_uri

data = {'data':
            [{
                "golddiffat10": -500,
                "xpdiffat10": -300,
                "golddiffat15": 2000,
                "xpdiffat15": 1600
            }]
        }

input_data = json.dumps(data)
headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, data, headers=headers)
print(response.text)

An unexpected internal error occurred. Encountered Exception: Traceback (most recent call last):
  File "/azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/python3.6/site-packages/flask/wrappers.py", line 74, in get_json
    rv = json.loads(data)
  File "/azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/python3.6/site-packages/flask/json/__init__.py", line 236, in loads
    return _json.loads(s, **kwargs)
  File "/azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/python3.6/json/__init__.py", line 367, in loads
    return cls(**kw).decode(s)
  File "/azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/python3.6/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [52]:

print(service.get_logs())

2021-01-27T02:33:09,757897420+00:00 - iot-server/run 
2021-01-27T02:33:09,758738126+00:00 - gunicorn/run 
2021-01-27T02:33:09,760446340+00:00 - rsyslog/run 
2021-01-27T02:33:09,762113954+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [28]:
service.delete()

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: predictor
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: predictor"
    }
}